In [ ]:
import pathlib
import sys

import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

sys.path.append(str(pathlib.PurePath('..', '..', 'src')))
from linear_geodesic_optimization.data import input_network

In [ ]:
path_graph = pathlib.PurePath('graph.graphml')
path_graph_optimal = pathlib.PurePath('graph_optimal_transport.graphml')
path_graph_incorrect = pathlib.PurePath('graph_incorrect.graphml')

In [ ]:
def mercator(longitude = None, latitude = None):
    """
    Given a longitude in [-180, 180] and a latitude in [-90, 90], return
    an (x, y) pair representing the location on a Mercator projection.
    Assuming the latitude is no larger/smaller than +/- 85
    (approximately), the pair will lie in [-0.5, 0.5]^2.
    """
    x = longitude / 360. if longitude is not None else None
    y = np.log(np.tan(np.pi / 4. + latitude * np.pi / 360.)) / (2. * np.pi) if latitude is not None else None

    x = float(x)
    y = float(y)

    if x is None:
        return y
    if y is None:
        return x
    return (x, y)

def get_sphere_point(latlong):
    """Convert spherical coordinates to Cartesian coordinates."""
    latlong = np.array(latlong) * np.pi / 180.
    return np.array([
        np.cos(latlong[1]) * np.cos(latlong[0]),
        np.sin(latlong[1]) * np.cos(latlong[0]),
        np.sin(latlong[0])
    ])

def get_spherical_distance(a, b):
    """Find the distance on the unit sphere between two unit vectors."""
    # The value of (a @ b) is clamped between -1 and 1 to avoid issues
    # with floating point
    if np.all(a == b):
        return np.float64(0.)
    return np.arccos(min(max(a @ b, -1.), 1.))

def get_GCL(latlong_a, latlong_b):
    """Find the great circle latency between two points on Earth in ms."""
    # The following are in meters and meters per second
    circumference_earth = 40075016.68557849
    radius_earth = circumference_earth / (2. * np.pi)
    c = 299792458.

    # Convert spherical coordinates to Cartesian coordinates
    p_a = get_sphere_point(latlong_a)
    p_b = get_sphere_point(latlong_b)

    # Special case for slightly improved numerical stability
    if np.all(p_a == p_b):
        return 0.

    # Compute the latency, which is the travel time at the rate of two
    # thirds the speed of light
    return 2. * 1000. * get_spherical_distance(p_a, p_b) * radius_earth \
        / (2. * c / 3.)

In [ ]:
def get_network_plot(
    graph,
    ax = None,
    weight_label='ricciCurvature', color_min=-2., color_max=2., colormap='RdBu'
):
    if ax is None:
        fig, ax = plt.subplots(1, 1, facecolor='#808080')
    else:
        fig = ax.get_figure()
    ax.set_aspect('equal')
    ax.axis('off')

    # Plot the edges
    for u, v, d in graph.edges(data=True):
        # Don't double-draw edges
        if u > v:
            continue

        weight = None
        if weight_label not in d:
            if (v, u) in graph.edges and weight_label in graph.edges[v, u]:
                weight = graph.edges[v, u][weight_label]
            else:
                continue
        else:
            if (v, u) in graph.edges and weight_label in graph.edges[v, u]:
                # For edges where an opposite exists, just draw the
                # average value
                weight = (d[weight_label] + graph.edges[v, u][weight_label]) / 2.
            else:
                weight = d[weight_label]

        color = mpl.colormaps[colormap]((weight - color_min) / (color_max - color_min))

        x_u, y_u = mercator(graph.nodes[u]['long'], graph.nodes[u]['lat'])
        x_v, y_v = mercator(graph.nodes[v]['long'], graph.nodes[v]['lat'])
        ax.plot([x_u, x_v], [y_u, y_v], color=color)

    # Plot the vertices
    for node, d in graph.nodes(data=True):
        # If trim_vertices is set, then only plot the vertices with
        # incident edges
        if graph[node]:
            x, y = mercator(graph.nodes[node]['long'],
                            graph.nodes[node]['lat'])
            ax.plot(x, y, '.', ms=4, color='green')

    return fig

In [ ]:
graph = nx.read_graphml(path_graph)
graph_optimal = nx.read_graphml(path_graph_optimal)
graph_no_throughput = input_network.compute_ricci_curvatures(graph.copy())
graph_incorrect = nx.read_graphml(path_graph_incorrect)

In [ ]:
fig, (ax_1, ax_2, ax_3, ax_4) = plt.subplots(4, 1, facecolor='#808080', dpi=250)

get_network_plot(graph_no_throughput, ax_1)
get_network_plot(graph_incorrect, ax_2)
get_network_plot(graph, ax_3)
get_network_plot(graph_optimal, ax_4)

ax_1.set_title('Original Method')
ax_2.set_title('Incorrect Implementation of New Method')
ax_3.set_title('New Method')
ax_4.set_title('New Method with Optimal Transport')

plt.show()

In [ ]:
x = []
y = []
for (u, v) in graph_no_throughput.edges:
    if 'ricciCurvature' in graph_no_throughput.edges[(u, v)] and 'ricciCurvature' in graph.edges[(u, v)]:
        x.append(graph_no_throughput.edges[(u, v)]['ricciCurvature'])
        y.append(graph.edges[(u, v)]['ricciCurvature'])

fig, ax = plt.subplots(1, 1)
ax.plot(x, y, 'b.')
ax.set_xlabel('Curvature using Original Method')
ax.set_ylabel('Curvature using New Method')
# ax.set_aspect('equal')
plt.show()

In [ ]:
x = []
y = []
for (u, v) in graph_incorrect.edges:
    if 'ricciCurvature' in graph_incorrect.edges[(u, v)] and 'ricciCurvature' in graph.edges[(u, v)]:
        x.append(graph_incorrect.edges[(u, v)]['ricciCurvature'])
        y.append(graph.edges[(u, v)]['ricciCurvature'])

fig, ax = plt.subplots(1, 1)
ax.plot(np.linspace(-6.5, 1., 2), np.linspace(-6.5, 1., 2), 'r--')
ax.plot(x, y, 'b.')
ax.set_xlabel('Curvature using Incorrect Implementation')
ax.set_ylabel('Curvature using New Method')
ax.set_aspect('equal')
plt.show()

In [ ]:
x = []
y = []
for (u, v) in graph.edges:
    if 'ricciCurvature' in graph.edges[(u, v)] and 'ricciCurvature' in graph_optimal.edges[(u, v)]:
        x.append(graph.edges[(u, v)]['ricciCurvature'])
        y.append(graph_optimal.edges[(u, v)]['ricciCurvature'])

fig, ax = plt.subplots(1, 1)
ax.plot(np.linspace(-6.5, 1., 2), np.linspace(-6.5, 1., 2), 'r--')
ax.plot(x, y, 'b.')
ax.set_xlabel('Curvature using New Method')
ax.set_ylabel('Curvature using New Method with Optimal Transport')
ax.set_aspect('equal')
plt.show()